# FigureYa107ChIPheatmap

小丫画图出品, 2019-6-21

欢迎关注“小丫画图”公众号，同名知识星球等你加入

小丫微信: epigenomics  E-mail: figureya@126.com

作者：赵龙，中科院遗传所在读博士

擅长：ChIP-seq，MNase-seq，ATAC-seq，HiC，ChIA-PET，GWAS分析，R语言。

兴趣：单细胞RNA-seq，ATAC-seq，机器学习相关。

小丫编辑校验

## 需求描述

复现这个图，左侧按ChIP-seq聚类，右侧ATAC-seq行的排序跟左侧对应一致。

![](example.png)

出自：<http://genesdev.cshlp.org/content/32/2/96.long>

对这篇文章的解读：<https://mp.weixin.qq.com/s/zmf0GMdGZqNxseFZKHooOw>

## 应用场景

不仅限于按ChIP-seq的顺序排ATAC-seq，还可以自己组合各种factor的ChIP-seq，进而发现规律。

例如：在转录因子结合位点，染色质开放程度是怎样的？组蛋白修饰的分布是怎样的？

场景一：手上有多组ChIP-seq/ATAC-seq数据，按一种数据的聚类来排序，从其他数据的分布中发现规律。

场景二：自己只测了一两个样品的ChIP-seq/ATAC-seq，只画一条多难看，可以再找些公共数据，按这个画法，能深挖出更多规律。

## 环境设置

下载并安装Anaconda发行版，https://www.anaconda.com/distribution/#download-section

用Jupyter notebook打开FigureYa107ChIPheatmap.ipynb文档

**用法参考这篇：**<https://mp.weixin.qq.com/s/G-CQhNEJBmMRuDe2kxND_w>

通过conda安装deeptools：


In [33]:
!conda install -c bioconda deeptools

## 输入数据的准备

如果你的ChIP-seq/ATAC-seq数据已经保存为bw文件，并且有相应的peak.bed文件，就可以跳过这步，直接进入“开始画图”。

### 数据下载

测序数据在发表文章时，作者会把数据上传到GEO/SRA数据库，GEO里除了原始的fastq文件外，往往还会提供bigwig（bw）文件、或bedGraph文件，以及peak.bed文件等等。

**哪些数据更靠谱？**推荐首选高影响因子的文章里的数据，快速查询高分数据的方式，可参考FigureYa104GEOmining。

例文的数据链接：<https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE107242>，点击Download中的(custom)，选择以下文件下载：

GSM2863128_N06_0.RPKM_nor.bw

GSM2863129_NO6_24.RPKM_nor.bw

GSM2863130_NO6_72.RPKM_nor.bw

GSM2863131_NO6_cd19pos.RPKM_nor.bw

GSM2863142_Input_NO14_24_mm9.bedGraph.gz

GSM2863147_EBF1_NO14_24_mm9.bedGraph.gz

GSM2863148_EBF1_NO14_72_mm9.bedGraph.gz

GSM2863149_EBF1_NO14_cd19pos_mm9.bedGraph.gz

### 转换bedgraph到bigwig文件

例文给出的ChIP-seq数据是bedgraph格式，需要转换成bigwig文件。用的是UCSC的bedGraphToBigWig，用法参考这里的Example 3：
<http://genome.ucsc.edu/goldenPath/help/bigWig.html>

下载工具：bedGraphToBigWig，根据自己的系统选择合适的版本

In [34]:
#我用wget下载，你也可以复制粘贴地址，直接从浏览器下载
!wget http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64/bedGraphToBigWig
!wget http://hgdownload.soe.ucsc.edu/admin/exe/macOSX.x86_64/bedGraphToBigWig
!chmod 775 bedGraphToBigWig

有可能需要添加环境变量

下载mm9的染色体长度的文件

In [35]:
!wget http://hgdownload.cse.ucsc.edu/goldenPath/mm9/bigZips/mm9.chrom.sizes

现在就可以把ChIP-seq的bedGraph文件转成bw了：

In [36]:
!for file in *.bedGraph; do sort -k1,1 -k2,2n $file > $file.sorted.bedGraph; bedGraphToBigWig $file.sorted.bedGraph mm9.chrom.sizes $file.bw; rm $file.sorted.bedGraph; done


### 获得peak.bed

可以跳过这步，直接用我跑好的allpeak.bed文件

文章没有提供每个样品的peak文件，找peak不是这里的重点，就不从mapping开始跑了。仅从文章提供的EBF1 ChIP-seq的bedgraph文件开始做，定义较高的位置为peak，作为示例文件。有点粗糙，但足够用于展示需求描述所需解决的问题。

In [37]:
!for file in GSM*_EBF1_*.bedGraph; do awk '$4> 80 {print}' $file > $file.peak.bed; done
# 通过conda安装bedtools
!conda install -c bioconda bedtools
#合并和排序
!cat *.peak.bed | sort -k1.1V -k2,2n -k3,3n| bedtools merge -i - > allpeak.bed

**附：**正常的做法是用MACS2从每个样本的bam文件里call出peak.bed，例如：

```bash
macs2 callpeak -c Input.bam -t treatment.bam -q 0.05 -f BAM -g mm -n treatment 2> treatment.macs2.log
```

## 开始画图-左侧ChIP-seq

每次运行computeMatrix大概需要10分钟，具体用法参见<https://deeptools.readthedocs.io/en/develop/content/tools/computeMatrix.html?highlight=computeMatrix>，下面是computeMatrix的主要参数说明：
--referencePoint center：中间对齐
-b 3000 -a 3000：前后距离
-R xx.bed：bed文件
-S Input.bw 24_EBF1.bw 72_EBF1.bw prob_EBF1.bw：准备的bigwig文件

plotHeatmap的具体用法参见<https://deeptools.readthedocs.io/en/develop/content/tools/plotHeatmap.html?highlight=plotHeatmap>，主要参数说明：
--zMax 150 ： colorbar的最大值，对应的还有最小值--zMin
这里有一个关键是--outFileSortedRegions这个参数。
这里输出的bed文件就是我们heatmap对应的文件。主要用于绘制后面ATAC的heatmap与之对应。

In [38]:
#计算矩阵
!computeMatrix reference-point --referencePoint center -b 3000 -a 3000 --missingDataAsZero --skipZeros -p 40 -R allpeak.bed -S GSM2863142_Input_NO14_24_mm9.bedGraph.bw GSM2863147_EBF1_NO14_24_mm9.bedGraph.bw GSM2863148_EBF1_NO14_72_mm9.bedGraph.bw GSM2863149_EBF1_NO14_cd19pos_mm9.bedGraph.bw -o nom.gz
#画图，并输出bed文件，用于画右侧ATAC-seq
!plotHeatmap -m nom.gz -out ChIP-pre.pdf --zMax 150 --kmeans 5 --outFileSortedRegions Heatmap1sortedRegions.bed --samplesLabel 0hrs 24hrs 72hrs pro-B

生成的图在ChIP-pre.pdf里，位于当前文件夹。可以看出最上面两个cluster里的信号不像peak，这是上面找peak的方式粗糙导致的，正常自己的peak文件中应该不会存在这种情况。我们把这些异常的peak删掉就好了：

In [43]:
#删除第一个和第二个cluster对应的peak
!grep -v cluster_1 Heatmap1sortedRegions.bed | grep -v cluster_2 > allpeak2.bed

然后再运行一次

In [44]:
#计算矩阵
!computeMatrix reference-point --referencePoint center -b 3000 -a 3000 --missingDataAsZero --skipZeros -p 40 -R allpeak2.bed -S GSM2863142_Input_NO14_24_mm9.bedGraph.bw GSM2863147_EBF1_NO14_24_mm9.bedGraph.bw GSM2863148_EBF1_NO14_72_mm9.bedGraph.bw GSM2863149_EBF1_NO14_cd19pos_mm9.bedGraph.bw -o nom.gz
#画图，并输出bed文件，用于画右侧ATAC-seq
!plotHeatmap -m nom.gz -out ChIP.pdf --zMax 150 --kmeans 5 --outFileSortedRegions Heatmap1sortedRegions.bed --samplesLabel 0hrs 24hrs 72hrs pro-B
#如果不想画average profile，就加上参数：--whatToShow "heatmap and colorbar" 

生成的图在ChIP.pdf里，位于当前文件夹。

ChIP.pdf蓝色最大值已经超过100，原文只有16，这与生成bigwig文件时bamCoverage --normalizeUsing RPKM或其他标准化方法（CPM, BPM, RPGC）有很大关系。同时还跟这一步用bin的大小有关。

这里我觉得主要是bin的大小的原因。--binSize 越大，这个值就越大。所以我估计原文画图的时候用的bin是比较小的。

原文没有提供bw文件，我们通过bedgraph转过来，这个bin就有点大，所以数值就偏大。

看后面ATAC-seq的大小跟文章就很一致，因为用的是文章提供的bigwig文件。

具体分析自己的数据时，左右两侧数据从头开始用同样的标准和参数处理就可以了。

## 开始画图-右侧ATAC-seq

computeMatrix里用到的-R为上面画ChIP-seq的图时输出的bed文件，里面是有顺序的。

plotHeatmap的主要参数：

--sortRegions keep ： 这个参数是一个关键，要保证和ChIP-seq的顺序是一致的。

--colorMap 'Greens'：颜色，deeptools还有很多颜色可以选，具体参考help。

--whatToShow ：这个是展示heatmap还是带有profile的heatmap还是其他。可以参考help。

--samplesLabel： 标签的label

In [45]:
#计算矩阵
!computeMatrix reference-point --referencePoint center -b 3000 -a 3000 --missingDataAsZero --skipZeros -p 40 -R Heatmap1sortedRegions.bed -S  GSM2863128_N06_0.RPKM_nor.bw GSM2863129_NO6_24.RPKM_nor.bw GSM2863130_NO6_72.RPKM_nor.bw GSM2863131_NO6_cd19pos.RPKM_nor.bw -o nom2.gz
#画图
!plotHeatmap -m nom2.gz -out ATAC.pdf --sortRegions keep --zMax 15 --colorMap 'Greens' --samplesLabel 0hrs 24hrs 72hrs pro-B
#如果不想画average profile，就加上参数：--whatToShow "heatmap and colorbar" 

生成的图在ATAC.pdf里，位于当前文件夹

## 后期处理

输出的pdf文件都是矢量图，可以在Illustrator等编辑器里打开，编辑图形、文字、组图。

In [2]:
import IPython
print(IPython.sys_info())

!jupyter nbconvert --to html FigureYa107ChIPheatmap.ipynb 

{'commit_hash': 'd774f565b',
 'commit_source': 'installation',
 'default_encoding': 'UTF-8',
 'ipython_path': '/anaconda3/lib/python3.7/site-packages/IPython',
 'ipython_version': '7.4.0',
 'os_name': 'posix',
 'platform': 'Darwin-18.2.0-x86_64-i386-64bit',
 'sys_executable': '/anaconda3/bin/python',
 'sys_platform': 'darwin',
 'sys_version': '3.7.3 (default, Mar 27 2019, 16:54:48) \n'
                '[Clang 4.0.1 (tags/RELEASE_401/final)]'}
[NbConvertApp] Converting notebook FigureYa107ChIPheatmap.ipynb to html
[NbConvertApp] Writing 290196 bytes to FigureYa107ChIPheatmap.html
